In [4]:
%load_ext autoreload
%autoreload 2
# %load_ext dotenv
# %dotenv /home/yangk/yizhou/.env


from thoughtsculpt.chatGPT_API import load_model
from thoughtsculpt.data.interestingness_dataset import load_dataset_from_disk
from thoughtsculpt.evaluation.content_evaluator import T5_Evaluator
from thoughtsculpt.model.improver import ContentImprover


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


/home/yangk/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
t5_model_name = "t5-base"
ckpt_dir = "../thoughtsculpt/ckpts/prediction_interestingness-contrastive-t5-base-add_star/checkpoint-2820"
content_evaluator = T5_Evaluator(t5_model_name, ckpt_dir=ckpt_dir) 


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
combined_ds = load_dataset_from_disk("../thoughtsculpt/datasets/outlines/interesting")
train_size = int(0.9 * len(combined_ds))
test_size = len(combined_ds) - train_size
train_ds = combined_ds.select(range(train_size))
val_ds = combined_ds.select(range(train_size, train_size + test_size))

In [7]:
val_ds[10]["original_outlines"]

['The girl realizes that the man is attempting to molest the child.',
 'The girl intervenes and successfully saves the child from the molester.',
 'The girl comforts the scared child and ensures their safety.',
 'The police agree to let the girl help and assign her a detective to work with.',
 'The girl undergoes training and learns investigative techniques from the detective.',
 'The girl and the detective start gathering information and evidence about the molester.',
 'The girl and the detective devise a plan to catch the molester, involving a bait operation.']

In [11]:
from thoughtsculpt.model.simulator import MCTS, DFS, ToT, COT
model_name = "gpt-3.5-turbo"#"gpt-4-0125-preview" #"gpt-3.5-turbo"
model = load_model(model_name=model_name, temp=0.7)
content_improver = ContentImprover(model=model, evaluator=None, solver_class=MCTS)


In [12]:
idx = 0
original_outlines = val_ds[idx]["original_outlines"]

In [16]:
new_outlines_lst = content_improver.improve(original_outlines, depth=2, continous=True)

The number of outlines in the candidate is less than the original (8 < 10)
The number of outlines in the candidate is less than the original (2 < 10)
The number of outlines in the candidate is less than the original (2 < 10)
The number of outlines in the candidate is less than the original (2 < 10)


In [18]:
new_outlines_lst[0]

['The Huxley brothers, Jake and Ethan, accidentally discover an ancient gateway hidden within Ryhope Wood that transports them to a mysterious realm where myths come alive.',
 'As they navigate through the realm, Jake and Ethan encounter a diverse array of mythical creatures that challenge not only their physical abilities but also their beliefs and values.',
 'Each brother faces personal trials tailored to their strengths and weaknesses, pushing them to confront their inner demons and grow as individuals.',
 'The line between reality and myth blurs as the brothers are tested by a series of unexpected events that shake their trust in each other and their understanding of the realm.',
 "Uncovering a hidden conspiracy in Ryhope Wood, Jake and Ethan must race against time to unravel the truth behind the realm's existence and their role in its fate.",
 "A mysterious figure emerges as a potential ally, but their motives remain ambiguous, adding a layer of suspense and intrigue to the brothe

In [19]:
new_outlines_lst[1]

['Brothers Jake and Ethan discover an ancient gateway in Ryhope Wood, leading them to a mysterious realm where myths and legends come to life.',
 'The realm challenges Jake and Ethan to face their deepest fears and insecurities, but also presents opportunities for growth and self-discovery as they interact with enigmatic mythical beings.',
 'Each brother undergoes a personal journey of trials and tribulations that test their beliefs and values, pushing them to confront their inner demons and evolve as individuals.',
 'The line between reality and fantasy blurs as Jake and Ethan encounter unexpected obstacles and adversaries that strain their bond and trust, adding layers of complexity to their relationship.',
 "Unraveling a hidden conspiracy within Ryhope Wood becomes a race against time for Jake and Ethan, as they uncover shocking truths about the realm's origins and their intertwined destinies.",
 'A mysterious figure emerges as a potential ally, but their true motives remain cryptic